In [ ]:
import torch
from reconstruction import AE
from datasets import MeshData
from utils import utils, DataLoader, mesh_sampling, sap
import numpy as np
import pyvista as pv
from skimage import measure
import ipywidgets
from IPython.display import display
import meshplot as mp
import os, sys
from math import ceil

In [ ]:
# Meshplot left an annoying print statement in their code. Using this context manager to supress it...
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [6]:
# Dot product on the first dimension of n-dimensional arrays x and y
def dot(x, y):
    return np.einsum('i..., i... -> ...', x, y)

# Signed distance functions from Inigo Quilez https://iquilezles.org/articles/distfunctions/
# You could implement the smooth minimum operation as well to compose shapes together for more complex situations
def sdf_sphere(x, radius):
    return np.linalg.norm(x, axis=0) - radius


def sdf_torus(x, radius, thickness):
    
    q = np.stack([np.linalg.norm(x[[0, 1]], axis=0) - radius, x[2]])
    return np.linalg.norm(q, axis=0) - thickness

# Crop an n-dimensional image with a centered cropping region
def center_crop(img, shape):
    start = [a // 2 - da // 2 for a, da in zip(img.shape, shape)]
    end = [a + b for a, b in zip(start, shape)]
    slices = tuple([slice(a, b) for a, b in zip(start, end)])
    return img[slices]

# Add noise to coordinates
def gradient_noise(x, scale, strength, seed=None):
    shape = [ceil(s / scale) for s in x.shape[1:]]
    if seed:
        np.random.seed(seed)
    scalar_noise = np.random.randn(*shape)
    scalar_noise = zoom(scalar_noise, zoom=scale)
    scalar_noise = center_crop(scalar_noise, shape=x.shape[1:])
    vector_noise = np.stack(np.gradient(scalar_noise))
    return vector_noise * strength


In [9]:
plot=None
@mp.interact(
    radius=(0, 0.3, 0.01), 
    thickness=(0.01, 0.1, 0.01), 
    noise_scale=(5, 25), 
    noise_strength=(0.0, 0.4, 0.05),
    seed=(1, 100)
)
def show(radius, thickness, noise_scale, noise_strength, seed):
    global plot
    global sdf
    coords = np.linspace(-1, 1, 100)
    x = np.stack(np.meshgrid(coords, coords, coords))
    x = x + gradient_noise(x, noise_scale, noise_strength, seed)
    sdf = sdf_torus(x, radius, thickness)
    verts, faces, normals, values = measure.marching_cubes(sdf, level=0)
    
    if plot is None:
        plot = mp.plot(verts, faces, return_plot=True)
    else:
        with HiddenPrints():
            plot.update_object(vertices=verts, faces=faces)
        display(plot._renderer)

FloatSlider(value=0.1, max=0.3, step=0.01)

FloatSlider(value=0.05, max=0.1, min=0.01, step=0.01)

IntSlider(value=10, max=25, min=5)

FloatSlider(value=0.2, max=0.4, step=0.05)

IntSlider(value=1, min=1)

Button(description='Show', style=ButtonStyle())